# Hotspots

Hotspots California map: 
- Wildifre-wildifreSmokePM2.5, 
- Heat-wildfire,
- Heat-PM2.5, 
- Wildifre-Heat-SmokePM2.5P

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [2]:
df = pd.read_parquet("outputs/hotspots_per_fips_rolling_wc.parquet") 

In [3]:
gdf = gpd.read_file('data/WesternUSCensusTract/CensusTract2020_WesternUS.shp')

In [4]:
gdf = gdf[['FIPS', 'POPULATION', 'STATE_ABBR']]

## Total counts (tracts, days)

In [7]:
total_census_tracts = len(df.GEOID.unique())
total_census_tracts

18108

In [8]:
total_days = len(df)
total_days = total_days / total_census_tracts
total_days

5479.0

In [9]:
total_years = 15

In [10]:
import plotting
hspts = plotting.get_hotspot_list()

## Get Exposure days per census tract per year

In [11]:
def get_exposure_per_census_per_year(data):
    # sum all the exposure days per coexposure
    exposure_day_counts = data.sum().reset_index()
    exposure_day_counts.columns = ["value", "days"]

    # divide the sum by the numbers of census tracts and the number of years
    exposure_day_counts["avg_census"] = round(exposure_day_counts["days"]/total_census_tracts/total_years, 2)
    # df_stats["percent"] = round(df_stats["days"]/total_days*100., 3)
    return exposure_day_counts


In [12]:
df_ecy = get_exposure_per_census_per_year(df[hspts])
df_ecy

,value,days,avg_census
0,wfday,115632,0.43
1,heatday,1560169,5.74
2,smoke_pm_non_zero,7224168,26.60
3,smoke_pm_gt_five,2419449,8.91
4,hw,2595,0.01
5,_hws,8391047,30.89
6,hs,464589,1.71
7,hws,1954,0.01
8,ws,43692,0.16
9,hs5,179849,0.66


## Get Avg Exposure Person Year

In [15]:
# Groupby operation
data = df[hspts + ['GEOID']]

# sum exposure days per GEOID (census tract (ct) number)
grouped_df = data.groupby('GEOID').sum()
df = gdf.merge(grouped_df, left_on="FIPS", right_on="GEOID")

In [16]:
df.head()

,FIPS,POPULATION,STATE_ABBR,wfday,heatday,smoke_pm_non_zero,smoke_pm_gt_five,hw,_hws,hs,...,hs5,hws5,ws5,heatday_2d,smoke_pm_non_zero_2d,wfday_2d,_hws_2d,hs_2d,hws_2d,ws_2d
0,04001942600,1549,AZ,2,92,406,69,0,475,25,...,6,0,0,139,614,4,702,55,0,0
1,04001942700,4491,AZ,9,92,437,68,0,505,29,...,8,0,0,139,667,17,754,60,0,9
2,04001944000,5348,AZ,43,30,411,61,0,462,7,...,0,0,1,49,627,71,701,14,0,32
3,04001944100,5495,AZ,13,92,421,58,0,488,30,...,10,0,0,136,642,24,728,56,0,18
4,04001944201,4021,AZ,0,92,346,57,0,413,25,...,8,0,0,136,545,0,631,50,0,0


In [17]:
# total population

df["POPULATION"].sum() 

76334073

In [20]:
def get_avg_exposure_person_year(data):
    for col in hspts:
        # multiply the number of exposure days by ct population and divide it by total years
        data[col] = round(data[col] * data["POPULATION"] / total_years, 2)

    # sum per each exposure
    exposure_person_day_per_year = grouped_df.sum().reset_index()
    exposure_person_day_per_year.columns = ["value", "exposure_person_day_per_year"]

    # new column for total exposure person day per year per ct
    exposure_person_day_per_year["exposure_person_day_per_year_per_ct"] = round(exposure_person_day_per_year["exposure_person_day_per_year"]/total_census_tracts, 2)
    return exposure_person_day_per_year

In [21]:
df_epy = get_avg_exposure_person_year(df)
df_epy

,value,exposure_person_day_per_year,exposure_person_day_per_year_per_ct
0,wfday,115632,6.39
1,heatday,1560169,86.16
2,smoke_pm_non_zero,7224168,398.95
3,smoke_pm_gt_five,2419449,133.61
4,hw,2595,0.14
5,_hws,8391047,463.39
6,hs,464589,25.66
7,hws,1954,0.11
8,ws,43692,2.41
9,hs5,179849,9.93


In [28]:
merged_df = df_ecy.merge(df_epy, on=["value"])
merged_df

,value,days,avg_census,exposure_person_day_per_year,exposure_person_day_per_year_per_ct
0,wfday,115632,0.43,115632,6.39
1,heatday,1560169,5.74,1560169,86.16
2,smoke_pm_non_zero,7224168,26.60,7224168,398.95
3,smoke_pm_gt_five,2419449,8.91,2419449,133.61
4,hw,2595,0.01,2595,0.14
5,_hws,8391047,30.89,8391047,463.39
6,hs,464589,1.71,464589,25.66
7,hws,1954,0.01,1954,0.11
8,ws,43692,0.16,43692,2.41
9,hs5,179849,0.66,179849,9.93


## Merge tables and add full exposure name

In [29]:
temp_dict = {}
hsptdict = plotting.get_hotspopt_dict()

# get full name of the exposure
for key in hsptdict.keys():
    temp_dict[key] = hsptdict[key]['title_map']

merged_df['value_full'] = merged_df['value'].replace(temp_dict)

In [30]:
merged_df = merged_df[["value_full", "avg_census", "exposure_person_day_per_year", "exposure_person_day_per_year_per_ct"]]

In [31]:
from tabulate import tabulate
# for col in merged_df.select_dtypes(include=['float64']).columns:
#     merged_df[col] = merged_df[col].apply(lambda x: '{:.2f}'.format(x))

markdown_table = tabulate(merged_df, headers='keys', tablefmt='pipe', showindex=False)
print(markdown_table)

| value_full                                                                      |   avg_census |   exposure_person_day_per_year |   exposure_person_day_per_year_per_ct |
|:--------------------------------------------------------------------------------|-------------:|-------------------------------:|--------------------------------------:|
| Total days of wildfire                                                          |         0.43 |                         115632 |                                  6.39 |
| Total days of heat (over 95th percentile over last 5 years)                     |         5.74 |                        1560169 |                                 86.16 |
| Total days of smoke $PM_{2.5}$ (non zero)                                       |        26.6  |                        7224168 |                                398.95 |
| Total days of smoke $PM_{2.5}$ (over >5μg/$m^3$)                                |         8.91 |                        2419449 |         

| value_full                                                                      |   avg_census |   exposure_person_day_per_year |   exposure_person_day_per_year_per_ct |
|:--------------------------------------------------------------------------------|-------------:|-------------------------------:|--------------------------------------:|
| Total days of wildfire                                                          |         0.43 |                         115632 |                                  6.39 |
| Total days of heat (over 95th percentile over last 5 years)                     |         5.74 |                        1560169 |                                 86.16 |
| Total days of smoke $PM_{2.5}$ (non zero)                                       |        26.6  |                        7224168 |                                398.95 |
| Total days of smoke $PM_{2.5}$ (over >5μg/$m^3$)                                |         8.91 |                        2419449 |                                133.61 |
| Total days of heat and wildfire concurrence                                     |         0.01 |                           2595 |                                  0.14 |
| Total days of heat or wildfire or smoke $PM_{2.5}$                              |        30.89 |                        8391047 |                                463.39 |
| Total days of heat and non-zero smoke $PM_{2.5}$ concurrence                    |         1.71 |                         464589 |                                 25.66 |
| Total days of heat, wildfire and non-zero smoke $PM_{2.5}$ concurrence          |         0.01 |                           1954 |                                  0.11 |
| Total days of wildfire & smoke $PM_{2.5}$                                       |         0.16 |                          43692 |                                  2.41 |
| Total days of heat and smoke $PM_{2.5}$ (over >5μg/$m^3$) concurrence           |         0.66 |                         179849 |                                  9.93 |
| Total days of heat, wildfire and smoke $PM_{2.5}$ (over >5μg/$m^3$) concurrence |         0    |                           1141 |                                  0.06 |
| Total days of wildfire and smoke $PM_{2.5}$ (over >5μg/$m^3$) concurrence       |         0.07 |                          19280 |                                  1.06 |
| Total days of heat (over 95th percentile over last 5 years) (2D)                |         8.66 |                        2350900 |                                129.83 |
| Total days of smoke $PM_{2.5}$ (non zero) (2D)                                  |        38.16 |                       10364963 |                                572.4  |
| Total days of wildfire (2D)                                                     |         0.73 |                         197017 |                                 10.88 |
| Total days of heat or wildfire or smoke $PM_{2.5}$ (2D)                         |        43.85 |                       11911409 |                                657.8  |
| Total days of heat and non-zero smoke $PM_{2.5}$ concurrence (2D)               |         3.37 |                         915543 |                                 50.56 |
| Total days of heat, wildfire and non-zero smoke $PM_{2.5}$ concurrence (2D)     |         0.02 |                           4755 |                                  0.26 |
| Total days of wildfire & smoke $PM_{2.5}$ (2D)                                  |         0.31 |                          84732 |                                  4.68 |

## State Stats

In [32]:
df.head()

,FIPS,POPULATION,STATE_ABBR,wfday,heatday,smoke_pm_non_zero,smoke_pm_gt_five,hw,_hws,hs,...,hs5,hws5,ws5,heatday_2d,smoke_pm_non_zero_2d,wfday_2d,_hws_2d,hs_2d,hws_2d,ws_2d
0,04001942600,1549,AZ,21327.66,981088.06,4329586.15,735816.31,0.0,5065402.46,266600.46,...,63984.03,0.0,0.00,1482296.96,6547698.38,42656.36,7486131.12,586520.59,0.0,0.00
1,04001942700,4491,AZ,806763.24,8246913.12,39172837.32,6095544.48,0.0,45268381.80,2599570.44,...,717122.88,0.0,0.00,12460010.04,59790120.12,1523886.12,67588831.44,5378421.60,0.0,806763.24
2,04001944000,5348,AZ,5465987.58,3813480.53,52244683.31,7754075.90,0.0,58727600.21,889810.94,...,0.00,0.0,127114.83,6228683.68,79701743.15,9025238.45,89108329.65,1779625.44,0.0,4067713.76
3,04001944100,5495,AZ,1744600.22,12346411.44,56498245.56,7783605.22,0.0,65489655.44,4026003.33,...,1341999.89,0.0,0.00,18251213.89,86156471.33,3220802.67,97697682.11,7515207.44,0.0,2415602.00
4,04001944201,4021,AZ,0.00,6611094.98,24863470.16,4096005.05,0.0,29678070.81,1796494.34,...,574877.01,0.0,0.00,9772925.23,39163556.20,0.00,45343495.43,3592986.00,0.0,0.00


In [33]:
# Groupby operation
data = df[hspts + ['POPULATION', 'STATE_ABBR']]

In [34]:
grouped_df = data.groupby('STATE_ABBR').sum()
for col in hspts:
    grouped_df[col] = round(grouped_df[col] * grouped_df["POPULATION"] / total_years, 2)

In [35]:
grouped_df

,wfday,heatday,smoke_pm_non_zero,smoke_pm_gt_five,hw,_hws,hs,hws,ws,hs5,hws5,ws5,heatday_2d,smoke_pm_non_zero_2d,wfday_2d,_hws_2d,hs_2d,hws_2d,ws_2d,POPULATION
STATE_ABBR,,,,,,,,,,,,,,,,,,,,
AZ,2.384958e+14,6.490316e+15,1.496066e+16,3.362186e+15,8.182064e+12,2.009406e+16,1.499998e+15,6.027137e+12,9.325662e+13,5.520425e+14,3.197088e+12,1.631317e+13,9.651119e+15,2.258357e+16,3.945359e+14,2.941854e+16,3.034383e+15,1.247735e+13,1.722403e+14,7151502
CA,9.211483e+15,2.244670e+17,8.781700e+17,3.022442e+17,3.410909e+14,1.052130e+18,5.684953e+16,2.511100e+14,2.779272e+15,2.225499e+16,8.780438e+13,1.163608e+15,3.347278e+17,1.274801e+18,1.665519e+16,1.508379e+18,1.112289e+17,6.737817e+14,6.429109e+15,39507131
CO,1.090665e+14,3.991488e+15,2.185820e+16,6.661344e+15,2.346665e+12,2.436473e+16,1.565144e+15,1.430418e+12,2.796669e+13,5.601431e+14,6.225749e+11,1.165989e+13,6.545140e+15,3.107932e+16,1.967107e+14,3.454634e+16,3.217101e+15,4.194083e+12,5.609068e+13,5773714
ID,9.403183e+13,4.103061e+14,3.081576e+15,1.343819e+15,1.577444e+12,3.340154e+15,2.065397e+14,1.349733e+12,3.899193e+13,9.404081e+13,8.484646e+11,1.582888e+13,6.260285e+14,4.157359e+15,1.571771e+14,4.464647e+15,3.985276e+14,3.647419e+12,7.681556e+13,1839106
MT,2.753128e+13,1.142294e+14,1.031118e+15,4.368499e+14,6.708793e+11,1.095765e+15,6.544622e+13,5.255360e+11,1.152221e+13,2.827760e+13,2.828572e+11,6.312095e+12,1.792809e+14,1.358837e+15,4.760401e+13,1.442511e+15,1.209836e+14,1.574345e+12,2.200042e+13,1084225
NM,3.703786e+13,4.925642e+14,1.812380e+15,3.987919e+14,1.077922e+12,2.187969e+15,1.435516e+14,7.230170e+11,1.010664e+13,3.238757e+13,1.888829e+11,2.377182e+12,7.191866e+14,2.734729e+15,6.443336e+13,3.219483e+15,2.771244e+14,1.876997e+12,2.114328e+13,2117522
NV,1.940530e+13,1.189263e+15,3.870704e+15,1.570506e+15,1.455672e+12,4.722053e+15,3.483278e+14,1.034201e+12,8.569851e+12,1.583182e+14,6.206401e+11,3.633120e+12,1.677321e+15,5.564867e+15,3.323241e+13,6.656823e+15,6.010213e+14,2.546836e+12,1.672113e+13,3104614
OR,2.626078e+14,2.241513e+15,1.361223e+16,4.082431e+15,5.810002e+12,1.522524e+16,7.948809e+14,4.340713e+12,9.476277e+13,2.879918e+14,3.033421e+12,3.892280e+13,3.404142e+15,1.930711e+16,4.622079e+14,2.132660e+16,1.638246e+15,1.058911e+13,2.061798e+14,4224980
TX,0.000000e+00,2.266899e+08,6.332535e+08,1.700175e+08,0.000000e+00,8.032710e+08,5.667244e+07,0.000000e+00,0.000000e+00,1.971211e+07,0.000000e+00,0.000000e+00,3.252509e+08,9.338643e+08,0.000000e+00,1.155626e+09,1.034889e+08,0.000000e+00,0.000000e+00,2026


In [36]:
grouped_df["POPULATION"].sum()

76334073